In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install undetected-chromedriver

In [93]:
# For webscraping
from bs4 import BeautifulSoup
# Parsing and creating xml data
from lxml import etree as et
# Store data as a csv file written out
from csv import writer
# In general to use with timing our function calls to Indeed
import time
# Assist with creating incremental timing for our scraping to seem more human
from time import sleep
# Dataframe stuff
import pandas as pd
# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint
# Multi Threading
import threading
# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import selenium

# Check version I am running
selenium.__version__

# Selenium 4:

from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [94]:
# Allows you to cusotmize: incognito mode, maximize window size, headless browser, disable certain features, etc
option= webdriver.ChromeOptions()

# Going undercover:
#option.add_argument("--incognito")


In [95]:
# Finding location, position, radius=35 miles, sort by date and starting page
paginaton_url = 'https://mx.indeed.com/jobs?q={}&l={}&radius=35&filter=0&sort=date&start={}&sc=0kf%3Acmpsec%28NKR5F%29%3B'

#For infocomm filter the code is sc=0kf%3Acmpsec%28NKR5F%29%3B
#https://mx.indeed.com/jobs?q={}&l={}&radius=35&filter=0&sort=date&start={}&sc=0kf%3Acmpsec%28NKR5F%29%3B'

In [97]:
start = time.time()

joblist_infocomm = [
    "Software Developer",
    "Systems Engineer", 
    "Data Analyst",
    "Data Scientist",
    "Network Engineer",
    "Web Developer",
    "Database Administrator",
    "IT Security Specialist",
    "Telecommunications Engineer",
    "Business Analyst",
    "BI Analyst (Business Intelligence)",
    "Business Analytics Consultant",
    "Business Analysis Specialist",
    "Cybersecurity Analyst",
    "Information Security Manager",  "Cybersecurity Consultant",
    "Penetration Tester",
    "Ethical Hacker",
    "Security Architect",
    "Network Security Engineer",
    "Information Assurance Analyst",
    "Incident Response Analyst",
    "Vulnerability Analyst",
    "Compliance Analyst",
    "Security Software Developer","IT Project Manager",
    "Technical Project Manager",
    "Digital Project Manager",
    "Agile Project Manager",
    "Scrum Master",
    "Program Manager",
    "Project Coordinator",
    "Project Analyst",
    "Portfolio Manager",
    "PMO Analyst (Project Management Office Analyst)",
    "Change Management Specialist",
    "Implementation Manager",
    "Project Lead",
    "Product Owner",
    "DevOps Engineer",
    "Cloud Architect",    
    "Systems Administrator",
    "IT Support Specialist",
    "Network Administrator",
    "IT Consultant",
    "Technical Support Engineer",
    "Cybersecurity Engineer",
    "IT Auditor",
    "Quality Assurance Analyst",
    "IT Trainer",
    "Network Operations Center (NOC) Technician"
]

joblist_infocomm = [
    "Systems Integration Engineer"
]

#location='San Francisco'
location='Mexico'

In [98]:

def scrap_job_infos(job_):
    job_title_list = []
    job_url_list = []
    job_id_list = []
    job_description_list_href=[]
    job_description_list = []
    salary_list=[]
    job_location_list = []
    type_of_employement_list = []

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
    sleep(1)
    driver.get(paginaton_url.format(job_,location,0))
    p=driver.find_element(By.CLASS_NAME,'jobsearch-JobCountAndSortPane-jobCount').text
    max_iter_pgs=int(p.split(' ')[0].replace(",", ""))//15 
    #max_page_iter = 'Max Iterable Pages for this search: {}'.format(max_iter_pgs) 
    # optional pagination below
        # driver.get("https://www.indeed.com/q-USA-jobs.html")
    if max_iter_pgs > 5:
        max_iter_pgs = 3
    elif max_iter_pgs > 2:
        max_iter_pgs = 3
    else:
        max_iter_pgs = 1
    for i in range(0,max_iter_pgs):
        try: 
            driver.get(paginaton_url.format(job_,location,i*10))
            sleep(randint(2, 4))
            job_page = driver.find_element(By.ID,"mosaic-jobResults")
            jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list
            for jj in jobs:
                job_title = jj.find_element(By.CLASS_NAME,"jobTitle")
        #           print(job_title.text)
                job_title_list.append(job_title.text)
                job_url_list.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"))
                job_id_list.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"))      
                    # jj.find_element(By.CLASS_NAME,"companyName").text,       
                    # jj.find_element(By.CLASS_NAME,"companyLocation").text,
                    # jj.find_element(By.CLASS_NAME,"date").text,
                    # job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
                try:
                    job_location_list.append(jj.find_element(By.CLASS_NAME, "company_location").text)
                
                except:
                    job_location_list.append("Job location not found")

                try:
                    type_of_employement_list.append(jj.find_element(By.XPATH, '//*[@id="mosaic-provider-jobcards"]/ul/li[4]/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[3]/div[2]/div').text)
                
                except:
                    type_of_employement_list.append("Employement type not found")

                try: # I removed the metadata attached to this class name to work!
                    salary_list.append(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)

                except NoSuchElementException: 
                    try: 
                        salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
                        
                    except NoSuchElementException:
                        salary_list.append("No salary found")
            
                        
        #         # Click the job element to get the description
                job_title.click()
                
        #         # Help to load page so we can find and extract data
                sleep(randint(2, 6))
                try: 
                    job_description_list.append(driver.find_element(By.ID,"jobDescriptionText").text)
                    
                except:
                    job_description_list.append("No description found")
        except:
            sleep(20)
        continue

    driver.quit()

    lst = list(zip(job_title_list, job_url_list, job_id_list, salary_list, job_description_list, type_of_employement_list, job_location_list)) 
    df = pd.DataFrame(lst, columns =['Title','URL', 'Job_ID', 'Salary', 'Description', 'Employement Type', 'Job Location'])
    return df

# alternate way to grab the info for job description to make it faster:

In [99]:
for job_ in joblist_infocomm:
    df = scrap_job_infos(job_)
    csv_path = "demo/{}_job_{}_June.csv".format(job_, location)
    df.to_csv(csv_path)
    end = time.time()
    print(end - start,'seconds to complete Query!')

362.5089988708496 seconds to complete Query!


In [ ]:
df.describe()

In [ ]:
df.head()

# Extract Skills and SoftSkills from the data

In [70]:
import os

In [43]:
#OPEN AI KEY

os.environ["OPENAI_API_KEY"] = "openai-api-key"


In [101]:
from langchain.chains import LLMChain


In [102]:
import os
import pandas as pd
import openai
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

OPENAI_API_KEY: str = os.environ.get("OPENAI_API_KEY")

In [103]:
df_skill = pd.read_csv(csv_path)


In [104]:
df_skill.head()

,Unnamed: 0,Title,URL,Job_ID,Salary,Description,Employement Type,Job Location
0,0,Salesforce QA Engineer,https://mx.indeed.com/rc/clk?jk=9beaa941cfabf4...,job_9beaa941cfabf4c2,No salary found,"About Us\nAt Cloudflare, we have our eyes set ...",Employement type not found,Cloudflare\nCiudad de México
1,1,Data Integration Engineer,https://mx.indeed.com/rc/clk?jk=1495d8f700e2a2...,job_1495d8f700e2a294,No salary found,DESCRIPTION\n\nJoin our dynamic team as a Data...,Employement type not found,"EPAM Systems\nFederal, CDMX"
2,2,Software Engineer - App Back-End (Remote - Mex...,https://mx.indeed.com/rc/clk?jk=e2989e6ae9a232...,job_e2989e6ae9a23294,No salary found,Summary\nYelp engineering culture is driven by...,Employement type not found,Yelp\nRemoto
3,3,L2 Production Support - ABO - GS,https://mx.indeed.com/rc/clk?jk=52b5191a54c7c2...,job_52b5191a54c7c2eb,No salary found,RH: Dacia Gonzalez\nLocation: any state of Mex...,Employement type not found,"Capgemini\n20116, Aguascalientes, Ags."
4,4,L2 Production Support - ABO - GS,https://mx.indeed.com/rc/clk?jk=6afcc51e4e2456...,job_6afcc51e4e2456fe,No salary found,RH: Dacia Gonzalez\n\nLocation: any state of M...,Employement type not found,"Capgemini\n20116, Aguascalientes, Ags."


In [105]:
job_skills = []

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature = 0)

template = """ From the given document: {document}
Question: {question}"""
prompt = PromptTemplate(template=template, input_variables=["question", "document"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

for x in df_skill["Description"]:
    if len(str(x)) > 3:
        question = "Provide a concise list of competences, skills, abilities, and knowledge mentioned in the text. Split lengthy items into smaller components. Your response should solely consist of a list of competences. If an item contains commas, break it down into separate entities."
        output_skills = llm_chain.run(question = question, document = x)
        job_skills.append(output_skills)
    else:
        print("description not found")
        job_skills.append("description not found")

In [106]:
job_skills

['\n\n1. Quality assurance\n2. Automation testing\n3. Salesforce.com platform\n4. Salesforce CPQ\n5. Quote-to-Cash system\n6. Test plan creation\n7. Test case creation\n8. Integration testing\n9. End-to-end system process testing\n10. Analytical skills\n11. Problem-solving abilities\n12. Agile/Scrum development process\n13. Communication skills\n14. Multi-tasking abilities\n15. Deadline management\n16. Flexibility\n17. Technical writing\n18. Software development\n19. Business analysis\n20. Project management\n21. DNS resolver\n22. Export control laws\n23. Diversity and inclusiveness\n24. Reasonable accommodations',
 '\n\n1. Proficiency in SQL for data manipulation and querying\n2. Experience with Data ETL processes\n3. Solid understanding of Data Integration and Data Warehousing principles\n4. Strong data analysis and visualization skills\n5. Familiarity with Google Workspace for collaborative work\n6. Ability to construct efficient data processing workflows and pipelines\n7. Experienc

In [51]:
len(df_skill["Description"])

3

In [108]:
df_skill['Skills'] = job_skills
df_skill['SoftSkills'] = job_softskills


In [109]:

combine_df_path = "demo/{}_completedataset_Feb_{}.csv".format(job_, location)

In [112]:
df_skill.head(10)

,Unnamed: 0,Title,URL,Job_ID,Salary,Description,Employement Type,Job Location,Skills,SoftSkills
0,0,Salesforce QA Engineer,https://mx.indeed.com/rc/clk?jk=9beaa941cfabf4...,job_9beaa941cfabf4c2,No salary found,"About Us\nAt Cloudflare, we have our eyes set ...",Employement type not found,Cloudflare\nCiudad de México,\n\n1. Quality assurance\n2. Automation testin...,\n\n1. Curiosidad\n2. Empatía\n3. Compromiso c...
1,1,Data Integration Engineer,https://mx.indeed.com/rc/clk?jk=1495d8f700e2a2...,job_1495d8f700e2a294,No salary found,DESCRIPTION\n\nJoin our dynamic team as a Data...,Employement type not found,"EPAM Systems\nFederal, CDMX",\n\n1. Proficiency in SQL for data manipulatio...,\n\n1. Colaboración\n2. Comunicación\n3. Creat...
2,2,Software Engineer - App Back-End (Remote - Mex...,https://mx.indeed.com/rc/clk?jk=e2989e6ae9a232...,job_e2989e6ae9a23294,No salary found,Summary\nYelp engineering culture is driven by...,Employement type not found,Yelp\nRemoto,\n\n1. Knowledge of Service Oriented Architect...,\n\n1. Cooperación en equipo\n2. Autenticidad ...
3,3,L2 Production Support - ABO - GS,https://mx.indeed.com/rc/clk?jk=52b5191a54c7c2...,job_52b5191a54c7c2eb,No salary found,RH: Dacia Gonzalez\nLocation: any state of Mex...,Employement type not found,"Capgemini\n20116, Aguascalientes, Ags.",\n\n1. Experience as a L2 support engineer\n2....,\n1. Trabajar bajo presión\n2. Calidad en el t...
4,4,L2 Production Support - ABO - GS,https://mx.indeed.com/rc/clk?jk=6afcc51e4e2456...,job_6afcc51e4e2456fe,No salary found,RH: Dacia Gonzalez\n\nLocation: any state of M...,Employement type not found,"Capgemini\n20116, Aguascalientes, Ags.",\n\n1. L2 support engineer experience\n2. ITIL...,\n\n1. Trabajar bajo presión\n2. Calidad en el...
5,5,Mid-Sr Full Stack Engineer (.NET & React),https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...,sj_525e24c5cb89e6cd,No salary found,Framework Science is on a MISSION that focuses...,Employement type not found,Framework Science\n4.3\nRemoto,\n\n1. C#\n2. .NET\n3. React\n4. Server-side l...,\n\n1. Comunicación\n2. Colaboración\n3. Resol...
6,6,Solution Architect/ Sr Data Engineer,https://mx.indeed.com/rc/clk?jk=bc664380222798...,job_bc66438022279865,No salary found,Solution Architect/ Sr Data Engineer\nArchitec...,Employement type not found,"UST\nGuadalajara, Jal.",\n\n1. Experiencia en desarrollo/arquitectura ...,\n\n1. Innovación\n2. Agilidad\n3. Empatía\n4....
7,7,Software Development Engineer,https://mx.indeed.com/rc/clk?jk=7134487f34c483...,job_7134487f34c483ca,No salary found,Minimum requirements for SW Design Verificatio...,Employement type not found,"Accenture\nMonterrey, N. L.",\n\n1. Bachelor's Degree in Computer Science o...,\n\n1. Habilidad para trabajar en equipo\n2. H...
8,8,Application Reliability Engineer - SAP Hybris,https://mx.indeed.com/rc/clk?jk=5f89835bdd3a0c...,job_5f89835bdd3a0c7b,No salary found,Application Reliability Engineer - SAP Hybris\...,Employement type not found,"Hewlett Packard Enterprise\nGuadalajara, Jal.",\n\n1. Troubleshooting\n2. Problem-solving\n3....,\n\n1. Accountability\n2. Action Planning\n3. ...
9,9,Site Reliability Developer,https://mx.indeed.com/rc/clk?jk=15854e41feb42a...,job_15854e41feb42ad7,No salary found,Solve complex problems related to infrastructu...,Employement type not found,"Oracle\nZapopan, Jal.",\n\n1. Automation\n2. Software engineering\n3....,\n\n1. Resolución de problemas complejos\n2. T...


# Add translated text


In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
import googletrans

In [ ]:
lang = detect_text_lang(df_skill["Skills"])
df_skill["lang"] = lang
to_span = translate_Eng_to_Span(df_skill["Skills"])
df_skill["Skills_es"] = to_span
to_span = translate_Eng_to_Span(df_skill["SoftSkills"])
df_skill["SoftSkills_es"] = to_span

In [149]:
df_skill.head()

,Unnamed: 0,Title,URL,Job_ID,Salary,Description,Employement Type,Job Location,Skills,SoftSkills
0,0,Auxiliar Administrativo Facturista,https://mx.indeed.com/rc/clk?jk=784fe286d283c8...,job_784fe286d283c817,"$9,000 a $9,500 por mes",Agencia de autos Volkswagen solicita Facturist...,Employement type not found,"MONTERREY MOTORS CUM\nMonterrey, N. L.",\n\n1. Manejo de Office\n2. Conocimientos de f...,\n\n1. Puntualidad\n2. Orden\n3. Proactividad\...
1,1,Analista de Inventarios,https://mx.indeed.com/rc/clk?jk=e7f43b63d0438a...,job_e7f43b63d0438acc,"Hasta $12,000 por mes","Importante Proveedora de Volkswagen solicita ""...",Employement type not found,"Servicios Logísticos\nSilao, Gto.",\n\n1. Manejo de Excel intermedio - avanzado\n...,\n\n1. Disponibilidad para rolar turnos\n2. Tr...
2,2,VENDEDOR (A) DE REFACCIONES,https://mx.indeed.com/rc/clk?jk=fae29809af2ad5...,job_fae29809af2ad5dc,"Desde $9,500 por mes",VOLKSWAGEN POPUL AUTO LLEGÓ A TEPIC!!\nÚNETE A...,Employement type not found,"POPUL AUTO TEPIC\n63130, Tepic, Nay.",\n\n1. Conocimiento en refacciones automotrice...,\n\n1. Atención al cliente\n2. Comunicación ve...
3,3,Técnico mecánico,https://mx.indeed.com/rc/clk?jk=09419c2c982e09...,job_09419c2c982e09f3,No salary found,Audi/ Porsche\nSolicita Técnico Mecánico Autom...,Employement type not found,"Audi Center\nSan Pedro Garza García, N. L.",\n\n1. Carrera técnica de mecánica\n2. Experie...,\n\n1. Disponibilidad de horario\n2. Trabajo e...
4,4,Diseñador Gráfico,https://mx.indeed.com/rc/clk?jk=e4467c647f1258...,job_e4467c647f12583a,"$9,000 por mes","Sueldo $9,000.00 netos\nBono de productividad\...",Employement type not found,"MEANING\nSan Andrés Cholula, Pue.",\n\n1. Licenciatura en un área afín a diseño g...,\n\n1. Trabajo en equipo\n2. Comunicación efec...


In [ ]:
df_skill.to_csv(combine_df_path)